In [75]:
import dask.dataframe as dd
import pandas as pd

In [76]:
# pd.set_option('display.max_columns', None)
# df = pd.read_parquet('../data/aggregated_data/parquet/tickets_full_merge_2016.parquet')
# df
ddf = dd.read_parquet('../data/aggregated_data/parquet/tickets_full_merge_2016.parquet')
ddf.head(5)

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Distance to CMS,Closest High School,Distance to CHS,Closest Individual Landmark,Distance to CIL,Closest Scenic Landmark,Distance to CIS,Closest Business,Industry of CB,Distance to CB
0,1363745293,KXD355,SC,PAS,1436400000000,21,SUBN,CHEVR,P,55730,...,1.117535,Brooklyn Academy High School,0.860486,679 Lafayette Avenue House (Magnolia Grandiflora),0.318490,Eastern Parkway,2.452519,FIRST STOP GROCERY INC. #2,Tobacco Retail Dealer,0.099806
1,1363745438,JCK7576,PA,PAS,1436400000000,21,SDN,ME/BE,P,42730,...,0.127059,Benjamin Banneker Academy,0.694492,Joseph Steele House,0.183244,Eastern Parkway,2.214997,243 DEKALB AVE. REST. CORP.,Sidewalk Cafe,0.044708
2,1363745475,GYK7658,NY,OMS,1436400000000,21,SUBN,NISSA,P,58130,...,0.963857,"Brooklyn Academy of Global Finance, The",0.869214,677 Lafayette Avenue House (Magnolia Grandiflora),0.631065,Eastern Parkway,2.962883,MAGALY GROCERY CORP.,Tobacco Retail Dealer,0.069942
3,1363745487,GMT8141,NY,PAS,1436400000000,21,P-U,LINCO,P,58130,...,0.737254,Brooklyn Academy High School,0.885687,677 Lafayette Avenue House (Magnolia Grandiflora),0.380512,Eastern Parkway,2.654091,BENSONS JEWELRY INC.,Secondhand Dealer - General,0.040407
4,1363745517,GYK3760,NY,PAS,1436400000000,21,SUBN,HONDA,P,46730,...,0.815431,"Brooklyn Academy of Global Finance, The",0.815431,William Ulmer Brewery Complex,0.385965,Eastern Parkway,3.207992,EL CHUNGO GROCERY CORP.,Tobacco Retail Dealer,0.148572


In [77]:
# https://stackoverflow.com/questions/46080171/constructing-mode-and-corresponding-count-functions-using-custom-aggregation-fun

def chunk(s):
    # for the comments, assume only a single grouping column, the 
    # implementation can handle multiple group columns.
    #
    # s is a grouped series. value_counts creates a multi-series like 
    # (group, value): count
    return s.value_counts()


def agg(s):
    # s is a grouped multi-index series. In .apply the full sub-df will passed
    # multi-index and all. Group on the value level and sum the counts. The
    # result of the lambda function is a series. Therefore, the result of the 
    # apply is a multi-index series like (group, value): count
    return s.apply(lambda s: s.groupby(level=-1).sum())

    # # faster version using pandas internals
    # s = s._selected_obj
    # return s.groupby(level=list(range(s.index.nlevels))).sum()


def finalize(s):
    # s is a multi-index series of the form (group, value): count. First
    # manually group on the group part of the index. The lambda will receive a
    # sub-series with multi index. Next, drop the group part from the index.
    # Finally, determine the index with the maximum value, i.e., the mode.
    level = list(range(s.index.nlevels - 1))
    return (
        s.groupby(level=level)
        # .apply(lambda s: s.reset_index(level=level, drop=True).argmax())
        .apply(lambda s: s.reset_index(level=level, drop=True).idxmax())
    )


mode = dd.Aggregation('mode', chunk, agg, finalize)
# mode = dd.Aggregation('mode', chunk, agg)

df_temp = pd.DataFrame({
    'col': [0, 1, 1, 2, 3] * 10,
    'g0': [0, 0, 0, 1, 1] * 10,
    'g1': [0, 0, 0, 1, 1] * 10,
})
ddf_temp = dd.from_pandas(df_temp, npartitions=10)

res = ddf_temp.groupby(['g0', 'g1']).agg({'col': mode}).compute()
print(res)

       col
g0 g1     
0  0     1
1  1     2


In [78]:
df_temp[(df_temp['g0'] == 0) & (df_temp['g1'] == 0)]['col'].mode()

0    1
Name: col, dtype: int64

In [79]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 60)

ddf.groupby('Issue Date').agg({'Registration State': mode}).compute().iloc[:100]

,Registration State
Issue Date,
1435795200000,NY
1435881600000,NY
1435968000000,NY
1436054400000,NY
1436140800000,NY
...,...
1444003200000,NY
1444089600000,NY
1444176000000,NY


In [80]:
mode = dd.Aggregation('mode', chunk, agg)
ddf.groupby('Issue Date').agg({'Registration State': mode}).compute().index.get_level_values(0).unique()

Index([1442880000000, 1456790400000, 1443657600000, 1444694400000,
       1441238400000, 1448582400000, 1444262400000, 1454544000000,
       1438819200000, 1457395200000,
       ...
       1464739200000, 1465430400000, 1464998400000, 1465689600000,
       1465862400000, 1464825600000, 1465516800000, 1464652800000,
       1465948800000, 1464393600000],
      dtype='int64', name='Issue Date', length=364)

In [81]:
mode = dd.Aggregation('mode', chunk, agg)
ddf.groupby('Issue Date').agg({'Registration State': mode}).compute().groupby(level=[0]).apply(lambda s: s.reset_index(level=[0], drop=True).idxmax())

,Registration State
Issue Date,
1435795200000,NY
1435881600000,NY
1435968000000,NY
1436054400000,NY
1436140800000,NY
...,...
1466812800000,NY
1466899200000,NY
1466985600000,NY


In [82]:
def chunk(s):
    # for the comments, assume only a single grouping column, the 
    # implementation can handle multiple group columns.
    #
    # s is a grouped series. value_counts creates a multi-series like 
    # (group, value): count
    return s.value_counts()


def agg(s):
    # s is a grouped multi-index series. In .apply the full sub-df will passed
    # multi-index and all. Group on the value level and sum the counts. The
    # result of the lambda function is a series. Therefore, the result of the 
    # apply is a multi-index series like (group, value): count
    return s.apply(lambda s: s.groupby(level=-1).sum())

    # # faster version using pandas internals
    # s = s._selected_obj
    # return s.groupby(level=list(range(s.index.nlevels))).sum()


def finalize(s):
    # s is a multi-index series of the form (group, value): count. First
    # manually group on the group part of the index. The lambda will receive a
    # sub-series with multi index. Next, drop the group part from the index.
    # Finally, determine the index with the maximum value, i.e., the mode.
    level = list(range(s.index.nlevels - 1))
    return (
        s.groupby(level=level)
        # .apply(lambda s: s.reset_index(level=level, drop=True).argmax())
        .apply(lambda s: s.reset_index(level=level, drop=True).idxmax())
    )


mode = dd.Aggregation('mode', chunk, agg, finalize)

In [83]:
ddf_mode = ddf.groupby('Issue Date').agg({'Registration State': mode, 'Plate Type': mode})
ddf_mode.compute()

,Registration State,Plate Type
Issue Date,,
1435795200000,NY,PAS
1435881600000,NY,PAS
1435968000000,NY,PAS
1436054400000,NY,PAS
1436140800000,NY,PAS
...,...,...
1466812800000,NY,PAS
1466899200000,NY,PAS
1466985600000,NY,PAS


In [84]:
ddf_other = ddf.groupby('Issue Date').agg({'temp': 'first', 'humidity': 'first', 'windspeed': 'first', 'visibility': 'first', 'conditions': 'first'})
ddf_other.compute()

,temp,humidity,windspeed,visibility,conditions
Issue Date,,,,,
1436400000000,23.1,72.4,16.6,11.6,"Rain, Partially cloudy"
1436054400000,24.0,60.3,14.8,15.0,Partially cloudy
1436832000000,24.3,74.2,15.1,14.2,"Rain, Partially cloudy"
1435881600000,23.0,52.0,20.5,15.8,Clear
1436918400000,24.7,76.0,16.8,10.7,"Rain, Partially cloudy"
...,...,...,...,...,...
1465516800000,19.7,33.8,23.4,16.0,Partially cloudy
1464652800000,24.6,67.9,14.6,14.3,Partially cloudy
1464393600000,27.0,57.0,19.6,13.7,Partially cloudy


In [85]:
ddf_joined = ddf_mode.merge(ddf_other, on='Issue Date')
ddf_joined.compute()

,Registration State,Plate Type,temp,humidity,windspeed,visibility,conditions
Issue Date,,,,,,,
1435795200000,NY,PAS,23.5,57.5,13.1,15.6,Partially cloudy
1435881600000,NY,PAS,23.0,52.0,20.5,15.8,Clear
1435968000000,NY,PAS,22.1,60.8,15.0,15.8,Partially cloudy
1436054400000,NY,PAS,24.0,60.3,14.8,15.0,Partially cloudy
1436140800000,NY,PAS,24.9,67.2,14.4,14.4,Partially cloudy
...,...,...,...,...,...,...,...
1466812800000,NY,PAS,22.9,56.1,15.8,15.3,Clear
1466899200000,NY,PAS,24.2,51.1,23.8,15.0,Partially cloudy
1466985600000,NY,PAS,23.6,61.3,29.3,14.4,"Rain, Partially cloudy"
